In [1]:
#Importing packages

import pandas as pd
import numpy as np
from sklearn import metrics
from sas7bdat import SAS7BDAT
import math
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import Logit

In [2]:
# if "No module named 'sas7bdat'" error occured, please install mentioned module, e.g. using below instruction

# !pip install sas7bdat

In [3]:
#Setting parameters

variable_name='Variable'
score_name_abt='SCORECARD_POINTS'
score_name="Score"
target_name='default12'    
time_report_name='Time'
time_report_name_up='TIME'
id_row='aid'
intercept_name='Intercept'
prob_event='PD'
event_rate_name='br'
event_rate_name_up='BR'
gini_train='Gini train'
gini_test='Gini test'
delta_gini='R. Gini'
INV='Information Value'
event_name='Bad'
all_name='All'
grp_name='Group'
nonevent_name='Good'
share_name='Share'
condition_name='Condition'
symbol_missing='Missing'
symbol_other='<OTHERS>'

In [4]:
#Functions definition

#Calibration funtion
def kal(x):
    r=1/(1+math.exp(-(model_cal.params[0]*x+model_cal.params[1])))
    if r<0: r=0
    if r>1: r=1
    return r

#Grouping - numeric variables
def grp_n(x):
    res=sub_l[grp_name][0]            
    for i in range(sub_l.shape[0]):
        sl=-np.inf
        sr=np.inf
        fl=sub_l[condition_name][i].find(' < ')
        fr=sub_l[condition_name][i].find(' <= ')
        if fl>=0:
            sl=float(sub_l[condition_name][i][0:fl])
        if fr>=0:
            sr=float(sub_l[condition_name][i][fr+3:])
        #fnm=condition_name][i].find(' <> '+symbol_missing)
        fm=sub_l[condition_name][i].find(' = '+symbol_missing)
        if fm>=0 and math.isnan(x):                
                res=sub_l[grp_name][i]            
        if fm<0 and (sl <= x < sr):                
                res=sub_l[grp_name][i] 
    return res

#Grouping - character variables
def grp_c(x):
    res=sub_l[grp_name][0] 
    if type(x) != str:
        x=str('')
    for i in range(sub_l.shape[0]):
        fo=sub_l[condition_name][i].find(symbol_other)
        if fo>=0:                
                res=sub_l[grp_name][i]            
    for i in range(sub_l.shape[0]):
        fo=sub_l[condition_name][i].find(symbol_other)
        if fo<0 and sub_l[condition_name][i].find(x)>=0:                
                res=sub_l[grp_name][i] 
    return res

In [5]:
#Data import

scorecard = pd.read_sas('scorecard_scorecard1.sas7bdat', encoding='LATIN2')
sce = pd.read_sas('scorecard_effects1.sas7bdat', encoding='LATIN2')
gmv = SAS7BDAT('good_models_valid.sas7bdat').to_data_frame()
gini_vars = pd.read_sas('variables_stat.sas7bdat', encoding='LATIN2')
abt = pd.read_sas('abt_score.sas7bdat', encoding='LATIN2')
abt[target_name] = abt[target_name].fillna(1)

In [6]:
#Data truncation, cleaning and manipulation - sce

sce = sce[sce.columns[~sce.columns.isin(['_ESTTYPE_'])]]
sce.rename(columns = {'DF':'Degrees of freedom',
                      'Estimate':'Estimation',
                      'StdErr':'Standard error', 
                      'WaldChiSq':'Wald Chi Sq',
                      'ProbChiSq':'Prob Chi Sq'}, inplace = True)

In [7]:
#List of choosen variables

features = list(sce[variable_name].str.replace('WOE_', ''))
features.remove("Intercept")

In [8]:
#Data truncation, cleaning and manipulation - gini_vars

gini_vars = gini_vars.loc[gini_vars['variable'].isin(features)]

scorecard.rename(columns = {'_variable_':variable_name}, inplace = True)
gini_vars.rename(columns = {'variable':variable_name}, inplace = True)
gini_vars = pd.merge(gini_vars, pd.DataFrame(scorecard[[variable_name, 'ivi']].groupby([variable_name]).agg({'ivi':'sum'})), on=variable_name)

gini_vars = gini_vars[[variable_name, 'AR_Train', 'AR_Valid', 'AR_Diff', 'ivi', 'H_GRP_TV', 'H_Br_GRP_TV', 'PR_Miss_Train', 'PR_Miss_Valid', 'N_Uni_Train', 'N_Uni_Valid', 'mfrequent_Train', 'mfrequent_Valid', 'PR_mfrequent_Train', 'PR_mfrequent_Valid', 'level']]
gini_vars.rename(columns = {'level':'Type',
                      'ivi': INV,
                      'H_GRP_TV': 'H GRP TV',
                      'H_Br_GRP_TV': 'H GRP TV BR',   
                      'PR_Miss_Train':'Missing percent train', 
                      'PR_Miss_Valid':'Missing percent valid',
                      'N_Uni_Train':'Number of distinct train',
                      'N_Uni_Valid':'Number of distinct valid',
                      'PR_mfrequent_Train':'P. mode train',
                      'PR_mfrequent_Valid':'P. mode valid',
                      'mfrequent_Train':'Mode train',
                      'mfrequent_Valid':'Mode valid',
                      'AR_Train':gini_train,
                      'AR_Valid':gini_test,
                      'AR_Diff':delta_gini}, inplace = True)

In [9]:
#Data truncation, cleaning and manipulation - scorecard

scorecard = pd.merge(scorecard, gini_vars[[variable_name, 'Type']], on=variable_name)
sce_nowoe = sce.copy()
sce_nowoe[variable_name].replace("WOE_", "", regex = True, inplace = True)
scorecard = pd.merge(scorecard, sce_nowoe[[variable_name, 'Estimation']], on=variable_name)

scorecard = scorecard[[variable_name, '_label_', 'br', '_percent_all_', '_number_all_', '_number_bad_', '_number_good_', 'logit', '_group_', 'Type', '_percent_bad_', '_percent_good_', 'ivi', 'Estimation', 'wi', 'SCORECARD_POINTS']]
scorecard.rename(columns = {'_label_':condition_name,
                            '_number_bad_':event_name, 
                            '_number_all_':all_name,
                            '_number_good_':nonevent_name,
                            'br':event_rate_name_up,
                            'logit':'Logit',
                            '_percent_all_':share_name,
                            '_percent_bad_':'Bad share',
                            '_percent_good_':'Good share',
                            'wi':"Weight of evidence",
                            'ivi':INV,
                            '_group_':grp_name,
                            'SCORECARD_POINTS':score_name}, inplace = True)

In [10]:
#Data truncation, cleaning and manipulation - gmv

gmv = gmv[['AR_Train', 'AR_Valid', 'AR_Diff', 'Min_WaldChiSq', 'Max_ProbChiSq', 'KS_Score', 'H_Score', 'H_Br_Score', 'Max_VIF_Train', 'Max_Vif_Valid', 'Max_Pearson_Train', 'Max_Pearson_Valid', 'Max_Con_Index_Train', 'Max_Con_Index_Valid', 'Lift1','Lift2','Lift3','Lift4','Lift5','Lift10','Lift20','Lift30','Lift40','Lift50', 'Gains1','Gains2','Gains3','Gains4','Gains5','Gains10','Gains20','Gains30','Gains40','Gains50']]
gmv.rename(columns = {'AR_Train':gini_train,
                      'AR_Valid':gini_test,
                      'AR_Diff':delta_gini, 
                      'Min_WaldChiSq':'Min WaldChiSq',
                      'Max_ProbChiSq':'Max ProbChiSq',
                      'KS_Score':'KS score',
                      'H_Score':'H score',
                      'H_Br_Score':'H br score',
                      'Max_VIF_Train':'Max VIF train',
                      'Max_Vif_Valid':'Max VIF valid',
                      'Max_Pearson_Train':'Max Pearson train',
                      'Max_Pearson_Valid':'Max Pearson valid',
                      'Max_Con_Index_Train':'Max Con Index train',
                      'Max_Con_Index_Valid':'Max Con Index valid'}, inplace = True)
gmv = gmv.T.reset_index()
gmv.columns=['Measure','Value']

In [11]:
#Data truncation, cleaning and manipulation - abt

abt['Time'] = abt['period'].str[:4]

In [12]:
#Gini over time

scored_all = abt.copy()
scored_all.dropna(subset=[target_name], inplace = True)

fpr, tpr, thresholds = metrics.roc_curve(scored_all[target_name], scored_all[score_name_abt])
gini=np.absolute(2*metrics.auc(fpr, tpr)-1)

scored=scored_all[[target_name, score_name_abt, time_report_name]].copy()
ulist=np.unique(scored[time_report_name])
time_gini=pd.DataFrame(ulist,columns=[time_report_name])
time_gini['Gini']=0.0
for t in range(len(ulist)):
    scored_sub=scored[scored[time_report_name]==ulist[t]].copy()
    fpr, tpr, thresholds = metrics.roc_curve(scored_sub[target_name], scored_sub[score_name_abt])
    time_gini['Gini'][t]=np.absolute(2*metrics.auc(fpr, tpr)-1)

C:\Users\piti1\AppData\Local\Temp\ipykernel_42640\3565931802.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_gini['Gini'][t]=np.absolute(2*metrics.auc(fpr, tpr)-1)
C:\Users\piti1\AppData\Local\Temp\ipykernel_42640\3565931802.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_gini['Gini'][t]=np.absolute(2*metrics.auc(fpr, tpr)-1)
C:\Users\piti1\AppData\Local\Temp\ipykernel_42640\3565931802.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

In [13]:
#Calibration

cal_scored=scored_all[[score_name_abt,target_name,time_report_name]].copy()
cal_scored[intercept_name]=1

features_c=[score_name_abt,intercept_name]
X=cal_scored[features_c]
y=cal_scored[target_name]

model_cal=sm.Logit(y,X).fit(disp=0,method='newton')

cal_scored[prob_event]=cal_scored[score_name_abt].apply(kal)
scored_all[prob_event]=scored_all[score_name_abt].apply(kal)

kalt=pd.DataFrame(cal_scored.groupby([time_report_name]).agg({target_name:['count','mean'], prob_event:['mean']}))
kalt=kalt.reset_index()
kalt.columns=[time_report_name,'ALL',event_rate_name_up,prob_event]

In [14]:
#Groups creation

varsc = list(abt.select_dtypes(include='object'))
varsc = [var.upper() for var in varsc]
varsn = list(abt.select_dtypes(include='number'))
varsn = [var.upper() for var in varsn]

In [15]:
#Model report
startrow=2

max_len = max((gini_vars[variable_name].astype(str).map(len).max(),len(str(gini_vars[variable_name].name)))) + 1 

writer = pd.ExcelWriter('Model_report.xlsx', engine='xlsxwriter')
book = writer.book


#Sheet - Main_measures
gmv.to_excel(writer, sheet_name='Main_measures', startrow=1, index_label=None, index=False)
sheet = writer.sheets['Main_measures']
sheet.set_column(0,0,15)


#Sheet - Effects
sce.to_excel(writer, sheet_name='Effects', startrow=1, index_label=None, index=False)

format2 = book.add_format({'num_format': '0.0000'})
sheet = writer.sheets['Effects']
sheet.set_column('F:F', None, format2)
sheet.set_column(0,0,25)


#Sheet - Gini_over_time
time_gini.to_excel(writer, sheet_name='Gini_over_time', startrow=1, index_label=None, index=False)

sheet = writer.sheets['Gini_over_time']
bold = book.add_format({'bold': True, 'size': 13})
sheet.write('A1', 'Gini all:', bold)
boldp = book.add_format({'bold': True, 'size': 13, 'num_format': '0.0%'})
sheet.write('B1', str(gini), boldp)
bold = book.add_format({'bold': True, 'size': 12})
boldp = book.add_format({'bold': True, 'size': 12, 'num_format': '0.0%'})
format1 = book.add_format({'num_format': '0.0%'})
sheet.set_column('B:B', None, format1)

ntimes=time_gini.shape[0]
shname='Gini_over_time'
chart = book.add_chart({'type': 'line'})
chart.set_title({'name': 'Gini over time'})
chart.set_x_axis({'name': '='+shname+'!A2', 'num_font':  {'rotation': 45}})
chart.add_series({'values': '='+shname+'!B3:B'+str(3+ntimes), 'name': '='+shname+'!B2',
                         'categories': '='+shname+'!A3:A'+str(3+ntimes)})
chart.set_legend({'position': 'bottom'})
sheet.insert_chart('D2', chart)


#Sheet - Scorecard
scorecard.to_excel(writer, sheet_name='Scorecard', startrow=1, index_label=None, index=False)
sheet = writer.sheets['Scorecard']
sheet.set_column(0,0,25)
sheet.set_column(1,1,50)


#Sheet - Variable importance
scale=pd.DataFrame(scorecard.groupby([variable_name]).agg({score_name:['min','max']}))
scale=scale.reset_index()
scale.columns=[variable_name,'Min score','Max score']
scale['Range']=scale['Max score']-scale['Min score']
ranges=scale['Max score'].sum()-scale['Min score'].sum()
scale['Importance']=scale['Range']/ranges
scale=scale.sort_values(by=['Importance'],ascending=False)
scale.to_excel(writer, sheet_name='Variable importance', startrow=1, index_label=None, index=False)
sheet = writer.sheets['Variable importance']
sheet.set_column(0,0,max_len)
sheet.set_column(1,1,len('Min score'))
sheet.set_column(2,2,len('Max score'))
sheet.set_column(3,3,len('Range'))
sheet.set_column(4,4,len('Variable importance'))


#Sheet - Calibration
kalt.to_excel(writer, sheet_name='Calibration', startrow=0, index_label=None, index=False)

sheet = writer.sheets['Calibration']
bold = book.add_format({'bold': True, 'size': 13})
sheet.write('F1', 'Score coeficient: '+str(model_cal.params[0]), bold)
sheet.write('F2', 'Intercept: '+str(model_cal.params[1]), bold)

sheet.write('F3', 'Formula: '+prob_event+'=1/(1+exp(-('+
            str(model_cal.params[0])+'*'+score_name+'+('
            +str(model_cal.params[1])+'))))', bold)


sheet.write('F5', event_rate_name_up+': '+str(cal_scored[target_name].mean()), bold)
sheet.write('F6', prob_event+': '+str(cal_scored[prob_event].mean()), bold)


#Sheet - Variable
gini_vars.to_excel(writer, sheet_name=variable_name, startrow=1, index_label=None, index=False)

sheet = writer.sheets[variable_name]
format1 = book.add_format({'num_format': '0.0%'})
sheet.set_column('B:B', None, format1)
sheet.set_column('C:C', None, format1)
sheet.set_column('D:D', None, format1)
sheet.set_column('G:G', None, format1)
sheet.set_column('H:H', None, format1)
sheet.set_column(0,0,max_len)
sheet.set_column(1,1,len(gini_train))
sheet.set_column(2,2,len(gini_test))
sheet.set_column(3,3,len(delta_gini))
sheet.set_column(4,4,15)
sheet.set_column(5,5,15)



#Set of sheets for variables
sss2=pd.DataFrame(abt[time_report_name].value_counts())
ntimes=sss2.shape[0]

abt.columns = abt.columns.str.upper()
time_report_name_up = time_report_name.upper()
target_name_up = target_name.upper()
    
for feature in features:    

    sssgr=pd.DataFrame(abt.groupby([time_report_name_up]).agg({target_name_up:['count']}))
    sssgr=pd.DataFrame(sssgr[target_name_up])
    sssgr=sssgr.reset_index()
    fin=pd.DataFrame()
    sss=pd.DataFrame()
    ddd=pd.DataFrame()
    sss=pd.DataFrame(abt.groupby(['PSC_'+feature,time_report_name_up]).agg({target_name_up:['sum','count']}))
    sss=pd.DataFrame(sss[target_name_up])
    sss=sss.reset_index()
    sss=sss.rename(columns={"sum": event_name, "count": all_name, 'PSC_'+feature:score_name})
    sss=pd.merge(sss, sssgr, on=time_report_name_up)
    sss[nonevent_name]=sss[all_name]-sss[event_name]
    sss[event_rate_name]=sss[event_name]/sss[all_name]
    sss[share_name]=sss[all_name]/sss["count"]
    sss=sss[[score_name,time_report_name_up,event_name,all_name,nonevent_name,event_rate_name,share_name]]
    sss=sss.sort_values(by=[score_name,time_report_name_up])

    sub=scorecard[scorecard[variable_name]==feature]
    # sub2=sub[[grp_name,score_name]]
    sub=sub[[score_name,condition_name,event_rate_name_up,share_name,all_name,event_name,nonevent_name]]
    sub_l=scorecard[scorecard[variable_name]==feature].reset_index()
    # if feature in varsn:
    #     sss[grp_name]=sss[grp_name].apply(grp_n)
    # elif feature in varsc:
    #     sss[grp_name]=sss[grp_name].apply(grp_c)
    # sss=pd.merge(sss, sub2, on=grp_name)
    sss=sss[[score_name,time_report_name_up,event_name,all_name,nonevent_name,event_rate_name,share_name]]
    ncat=sub.shape[0]

    sub.rename(columns = {'SCORECARD_POINTS':score_name,
                          '_label_':condition_name,  
                          'br':event_rate_name_up,
                          '_percent_all_':share_name, 
                          '_number_all_':all_name,
                          '_number_bad_':event_name,
                          '_number_good_':nonevent_name}, inplace = True)
    sss.rename(columns = {'SCORECARD_POINTS':score_name,
                          'TIME':time_report_name,
                          '_label_':condition_name,  
                          'br':event_rate_name_up,
                          '_percent_all_':share_name, 
                          '_number_all_':all_name,
                          '_number_bad_':event_name,
                          '_number_good_':nonevent_name}, inplace = True)


    shname=feature[0:31]
    sub.to_excel(writer, sheet_name=shname, startrow=startrow, index_label=None, index=False)
    sss.to_excel(writer, sheet_name=shname, startrow=startrow, index_label=None, index=False, startcol=8)
    sheet = writer.sheets[shname]
    bold = book.add_format({'bold': True, 'size': 24})
    sheet.write('A1', variable_name+': '+feature, bold)
    bold = book.add_format({'bold': True, 'size': 12})
    boldp = book.add_format({'bold': True, 'size': 12, 'num_format': '0.0%'})
    sheet.write_formula('D'+str(startrow+ncat+2), '=SUM(D4:D'+str(startrow+ncat+1)+')', boldp)
    sheet.write_formula('E'+str(startrow+ncat+2), '=SUM(E4:E'+str(startrow+ncat+1)+')', bold)
    sheet.write_formula('F'+str(startrow+ncat+2), '=SUM(F4:F'+str(startrow+ncat+1)+')', bold)
    sheet.write_formula('G'+str(startrow+ncat+2), '=SUM(G4:G'+str(startrow+ncat+1)+')', bold)

    format1 = book.add_format({'num_format': '0.0%'})
    sheet.set_column('C:C', None, format1)
    sheet.set_column('D:D', None, format1)
    sheet.set_column('N:N', None, format1)
    sheet.set_column('O:O', None, format1)

    series = sub[condition_name]
    max_len = max((series.astype(str).map(len).max(),len(str(series.name)))) + 1
    sheet.set_column(1,1,max_len)

    chart = book.add_chart({'type': 'line'})
    chart.set_title({'name': event_rate_name_up})
    chart.set_x_axis({'name': '='+shname+'!J3', 'num_font':  {'rotation': 45}})
    for i in range(ncat):
        chart.add_series({'values': '='+shname+'!N'+str(4+i*ntimes)+':N'+str(4+(i+1)*ntimes-1), 'name': '='+shname+'!I'+str(4+i*ntimes),
                         'categories': '='+shname+'!J'+str(4)+':J'+str(4+ntimes-1)})
    chart.set_legend({'position': 'bottom'})
    sheet.insert_chart('A'+str(ncat+6), chart)

    chart = book.add_chart({'type': 'line'})
    chart.set_title({'name': share_name})
    chart.set_x_axis({'name': '='+shname+'!J3', 'num_font':  {'rotation': 45}})
    for i in range(ncat):
        chart.add_series({'values': '='+shname+'!O'+str(4+i*ntimes)+':O'+str(4+(i+1)*ntimes-1), 'name': '='+shname+'!I'+str(4+i*ntimes),
                         'categories': '='+shname+'!J'+str(4)+':J'+str(4+ntimes-1)})
    chart.set_legend({'position': 'bottom'})
    sheet.insert_chart('A'+str(ncat+6+18), chart)

writer.save()